
## Desafio PS Estágio Recomb
### Rebeca Cecco 

<p>Descrição da solução : </p> 

*   Leitura dos arquivos de notas fisicas utilizando os metódos disponibilizados pela biblioteca nfelib com tratamento de certos tipos de erros (mais detalhes sobre os tipos nos comentários do código)
*   Persitência de dados e geração de relatórios foram implementados utilizando o SQlite3

<p> Execução do Programa: </p>

* Ultima célula de código contém a chamada de todos os metódos necessários para exibir as funcionalidades presentes na solução
* Existem duas opções de execução: Leitura de notas e geração de relatórios
* Na opção relatórios é preciso escolher qual tipo e inserir o identificador do fornecedor
* Se for escolhida uma opção de execução ou de relatório não esperada é exibida uma mensagem de erro e a execução acaba
* Na geração de relatórios se for fornecido um CPNJ inválido a execução também é encerrada

<p> Testes: </p>

* Cada tipo de erro tratado foi testado
* Erros previstos: Erros de Parser(mais detalhes nos comentários), arquivo inválido, erro de digitação de CPNJ para gerar relatórios, inserir nota já cadastrada
* Nome dos arquivos de teste seguem o padrão nfe_erro_{nome_do_tipo_de_erro}
* Caso de teste além do erro: cliente Pessoa Física

<p> Observações

* Comportamento não desejado da biblioteca: Imprime o xml após o parsing, na documentação disponível não consta como impedir isso

In [1]:
import sqlite3
import xml.etree.ElementTree as ET
import datetime
import re

! pip install git+https://github.com/akretion/nfelib.git@master_gen_v4_00#egg=nfelib

  Cloning https://github.com/akretion/nfelib.git (to revision master_gen_v4_00) to /tmp/pip-install-xx6awzrs/nfelib_2f0c866ca73e4d07bda428ef1da931a3
  Running command git clone -q https://github.com/akretion/nfelib.git /tmp/pip-install-xx6awzrs/nfelib_2f0c866ca73e4d07bda428ef1da931a3


In [2]:
from nfelib.v4_00 import leiauteNFe_sub as parser

In [3]:
#O desafio é desenvolver um programa que permita realizar as seguintes buscas:

#Listar os valores e data de Vencimento dos boletos presentes em um nota fiscal conforme o CPF ou CNPJ de um fornecedor.
#Apresentar o nome, identificador (CPF ou CNPJ), endereço dos clientes de um fornecedor.

In [4]:
#Metódo que trata erros de parser do tipo:
# 1- Falta de nome de cliente ou do fornecedor
# 2- Falta de CNPJ ou CPF do cliente ou do fornecedor
# 3- Endereço incompleto do cliente

#Interessante implementar verificações mais sofisticadas de erro
# validar tanto a existência quanto corretude de CPF, CEP e CNPJ por exemplo

def tratamento_erros_parser(nota_fiscal):
  #Pessoa Física não pode emitir nota por isso o programa só espera CNPJ do fornecedor

  #Tipo 1 e 2 de Erro para caso do fornecedor
  if nota_fiscal.infNFe.emit.CNPJ == "" or nota_fiscal.infNFe.emit.xNome == "":
    print("***** Erro na Leitura da Nota Fiscal de ID = {} *****\n Dados do Fornecedor faltando ".format(nota_fiscal.infNFe.Id))
    return True
  #Tipo 1 e 2 de Erro para o caso de Cliente
  elif nota_fiscal.infNFe.dest.CNPJ == "" or nota_fiscal.infNFe.dest.CPF == "" or nota_fiscal.infNFe.dest.xNome == "":
    print("***** Erro na Leitura da Nota Fiscal de ID = {} *****\n Dados do Cliente faltando ".format(nota_fiscal.infNFe.Id))
    return True
  #Tipo 3 de erro
  elif nota_fiscal.infNFe.dest.enderDest.CEP == "" or nota_fiscal.infNFe.dest.enderDest.xLgr == "" or nota_fiscal.infNFe.dest.enderDest.nro == "" :
    print("***** Erro na Leitura da Nota Fiscal de ID = {} *****\n Dados Endereço do Cliente Incompleto ".format(nota_fiscal.infNFe.Id))
    return True
  else :
    print("***** Leitura concluída com sucesso *****")
    return False


In [16]:
# Metódo de Leitura dos dados necessários para gerar os relatórios pedidos

def parser_notas(arquivo_nota):
  erro = True
  #metódo de parser da biblioteca
  nota_fiscal = parser.parse(arquivo_nota)
  erro = tratamento_erros_parser(nota_fiscal)
  
  if (not erro) : 
    id_nota = nota_fiscal.infNFe.Id
    atributos_fornecedor = [nota_fiscal.infNFe.total.ICMSTot.vNF, 
                            #a bib tem um comportamento diferente para estruturas de aninhamento similares
                            #para a tag dub os filhos são representados por lista
                            #por isso não é possível fazer o acesso padrão com .
                            nota_fiscal.infNFe.cobr.dup[0].dVenc,
                            nota_fiscal.infNFe.emit.CNPJ,
                            nota_fiscal.infNFe.emit.xNome]
    if nota_fiscal.infNFe.dest.CNPJ == None :
      atributos_cliente = [nota_fiscal.infNFe.dest.CPF, nota_fiscal.infNFe.dest.xNome]
    else:
      atributos_cliente = [nota_fiscal.infNFe.dest.CNPJ, nota_fiscal.infNFe.dest.xNome]
    endereco_cliente = [nota_fiscal.infNFe.dest.enderDest.xLgr,
                      nota_fiscal.infNFe.dest.enderDest.nro,
                      nota_fiscal.infNFe.dest.enderDest.xBairro,
                      nota_fiscal.infNFe.dest.enderDest.xMun,
                      nota_fiscal.infNFe.dest.enderDest.UF,
                      nota_fiscal.infNFe.dest.enderDest.CEP]
    atributos_cliente = atributos_cliente + endereco_cliente

    #escolha de projeto fazer como tupla e não lista de acordo com a espeficição do problema
    #afinal está sendo feito apenas uma leitura, não há necessidade de alteração e nem deveria ser possível
    nota_lida = ((id_nota),(atributos_fornecedor),(atributos_cliente))
    return nota_lida
  else: return False



In [6]:
#Checar se a Tabela está vazia

def checar_tabela(cur):
    cur.execute('''SELECT COUNT(*) from nota_fiscal ''')
    resultado = cur.fetchall()
    
    # se não há nehuma linha como resultado da query significa que a tabela está vazia
    if resultado == 0 :
        print("Não existem notas fiscais para gerar o relatório")
        return True
    else :
        return False

In [7]:
def impressao_relatorio(opcao_rel,cnpj):

  if(opcao_rel == "1"):
    print("Imprimindo Boletos do Fornecedor com CPNJ: {}\n".format(cnpj))
    linhas_rel = listar_boletos(cnpj) 
  else: 
    print("Imprimindo clientes do Fornecedor com CPNJ: {} \n",format(cnpj))
    linhas_rel = listar_clientes(cnpj)

  

In [8]:
#Listar os valores e data de Vencimento dos boletos presentes em um nota fiscal conforme o CPF ou CNPJ de um fornecedor.
def listar_boletos(cnpj):
  con = sqlite3.connect('nfe.db')
  cur = con.cursor()

  tabela_vazia = checar_tabela(cur)

  if (not tabela_vazia):
    for linha in cur.execute("SELECT valor, dataVencimento FROM nota_fiscal WHERE CNPJForncedor=? ", (cnpj,)) :
      print(linha)
      print("Valor Boleto: {} Data de Vencimento: {} \n".format(linha[0],linha[1]))

    qtd_linhas = cur.fetchall()
    if len(qtd_linhas) == 0:
      print("Não há notas desse Fornecedor na base de dados")

  con.close()

In [9]:
#Apresentar o nome, identificador (CPF ou CNPJ), endereço dos clientes de um fornecedor.
def listar_clientes(cnpj):
  con = sqlite3.connect('nfe.db')
  cur = con.cursor()

  tabela_vazia = checar_tabela(cur)

  if (not tabela_vazia):
    for linha in cur.execute("SELECT nomeCliente, idCliente, enderecoCliente FROM nota_fiscal WHERE CNPJForncedor=? ", (cnpj,)) :
      print("Nome Cliente: {}, Identificador do Cliente: {}, Endereço Cliente: {}".format(linha[0],linha[1],linha[2]))

    qtd_linhas = cur.fetchall()
    if len(qtd_linhas) == 0:
      print("Não há notas desse Fornecedor na base de dados")
  
  con.close()

In [24]:
def persistir_dados(nota_lida):
  id_nota = nota_lida[0]
  #retirar o "NFe" do começo do Id para que a chave seja númerica (idealmente)
  #um problema: da esse erro "Python int too large to convert to SQLite INTEGER"
  id_nota = id_nota[3:]
  atr_for = nota_lida[1]
  atr_cli = nota_lida[2]
  #removendo o nome e o cpf ou cnpj do cliente dos atributos pra concatenar as infos de endereço
  end_cli = atr_cli[2:]
  
  #transformando o endereço em uma string unica
  #idealmente fazer uma tabela endereço e guardar cada campo separadamente
  #como não havia nenhum requisito de busca por endereço fiz a simplificação
  endereco = end_cli[0] + "," + end_cli[1] + "," + end_cli[2] + "," + end_cli[3] + "," + end_cli[4] + "," +  end_cli[5]
  con = sqlite3.connect('nfe.db')

  cur = con.cursor()
  #cur.execute("drop table nota_fiscal")
  cur.execute ('''CREATE TABLE IF NOT EXISTS nota_fiscal
                        (idNota text, valor real, dataVencimento text, CNPJForncedor text, nomeFornecedor text, idCliente text, nomeCliente text, enderecoCliente text)''')
  #Verificação se já a nota já foi inserida anteriormente
  cur.execute("SELECT idNota=? FROM nota_fiscal",(id_nota,))
  nota = cur.fetchall()
  if(len(nota) > 0): print("Erro Nota Fiscal já cadastrada")
  else:
    cur.execute("INSERT INTO nota_fiscal VALUES (?,?,?,?,?,?,?,?) ",(id_nota, #id
                                                              (float (atr_for[0])), #valor do boleto
                                                              atr_for[1], #data de Vencimento
                                                              atr_for[2], #CNPJ Fornecedor
                                                              atr_for[3], #Nome do Fornecedor
                                                              atr_cli[0], #CPF ou CNPJ Cliente
                                                              atr_cli[1], #nome Cliente
                                                              endereco) )
    con.commit()
  
  con.close()


In [12]:
def checar_cnpj(cnpj):
  valido = False

  #checando se não houve erro de digitação e o cnpj possui algum caracter não númerico
  valido = str.isdigit(cnpj)

  
  if(valido):
    #checando se não existe algum digito faltando no cnpj
    if(len(cnpj) == 14) : return True
    else: return False
  else:
    return False
  #retonei os valores booleanos literais pra ficar mais claro 

In [13]:
def checar_arquivo(nome_arquivo):
  caminho_valido = re.match('^((\.\.|[a-zA-Z0-9_/\-\\ ])*\.[a-zA-Z0-9]+)$', nome_arquivo)
  
  if(caminho_valido):
    return caminho_valido
  else:
    return caminho_valido

In [25]:
####### CÉLULA PRINCIPAL DE CÓDIGO #######
#To do: fazer o programa rodar até que fosse digitado uma opção válida e não matar a execução

tipo_exec = input("Modos de Execução \n 1 - Leitura de NFe \n 2 - Relatórios \n")

#Execução de Leitura de Notas
if tipo_exec == "1":
  arquivo_nfe = input("Entre com Nome do Arquivo \n")
  if(checar_arquivo(arquivo_nfe) != None):
    nota_lida = parser_notas(arquivo_nfe)
    #se nota_lida for uma tupla ao inves de False guarda no BD
    if(nota_lida):persistir_dados(nota_lida)
  else:
    print("Arquivo inválido")

#Execução de relatórios
elif tipo_exec == "2":
  relatorio = input("Relatórios disponíveis: \n 1 - Boletos de um Fornecedor \n 2 - Clientes de um Fornecedor \n")
  if relatorio != "1" and relatorio != "2":
    print("Tipo de Relatório não suportado\n")
  else:
    fornecedor = input("Entre com CPNJ do Fornecedor \n")
    cnpj_valido = checar_cnpj(fornecedor)
    if(cnpj_valido):impressao_relatorio(relatorio,fornecedor)
    else:
      print("CNPJ inválido")
  
else: 
  print("Tipo de execução não suportado\n") 
    


Modos de Execução 
 1 - Leitura de NFe 
 2 - Relatórios 
1
Entre com Nome do Arquivo 
/content/drive/MyDrive/Colab Notebooks/nfe_cliente_pf.xml
<?xml version="1.0" ?>
<nfeProc xmlns="http://www.portalfiscal.inf.br/nfe"                          versao="4.00">
<NFe xmlns="http://www.portalfiscal.inf.br/nfe">
    <infNFe versao="4.00" Id="NFe31190406273476000182550020000031031004640327">
        <ide>
            <cUF>31</cUF>
            <cNF>00464032</cNF>
            <natOp>Vendas a prazo</natOp>
            <mod>55</mod>
            <serie>2</serie>
            <nNF>3103</nNF>
            <dhEmi>2019-04-10T17:24:03-02:00</dhEmi>
            <dhSaiEnt>2019-04-11T17:17:30-02:00</dhSaiEnt>
            <tpNF>1</tpNF>
            <idDest>1</idDest>
            <cMunFG>3170206</cMunFG>
            <tpImp>1</tpImp>
            <tpEmis>1</tpEmis>
            <cDV>7</cDV>
            <tpAmb>1</tpAmb>
            <finNFe>1</finNFe>
            <indFinal>0</indFinal>
            <indPres>1</indP